In [1]:
%pip install beautifulsoup4 requests pandas -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
df = pd.read_csv('AllAnimes.csv')
df.sample(5)

,position,title,episodes,release_date,members,score,details
45,46,Odd Taxi,TV (13 eps),Apr 2021 - Jun 2021,346548,8.71,https://myanimelist.net/anime/46102/Odd_Taxi
38,39,Hajime no Ippo,TV (75 eps),Oct 2000 - Mar 2002,506160,8.75,https://myanimelist.net/anime/263/Hajime_no_Ippo
43,44,Made in Abyss: Retsujitsu no Ougonkyou,TV (12 eps),Jul 2022 - Sep 2022,306775,8.72,https://myanimelist.net/anime/41084/Made_in_Ab...
28,29,Mob Psycho 100 II,TV (13 eps),Jan 2019 - Apr 2019,1324032,8.81,https://myanimelist.net/anime/37510/Mob_Psycho...
10,11,Ginga Eiyuu Densetsu,OVA (110 eps),Jan 1988 - Mar 1997,291686,9.03,https://myanimelist.net/anime/820/Ginga_Eiyuu_...


In [4]:
def get_details(details_page):
    details = [np.nan for i in range(7)]
    for i in details_page.find_all('div', {'class': 'spaceit_pad'}):
        if 'Broadcast:' in i.text:
            details[0] = str(i.text.split()[1:])
            pass
        if 'Studios:' in i.text:
            details[1] = i.find('a').text
            pass
        if 'Source:' in i.text:
            details[2] = i.text.split()[1].strip()
            pass
        if 'Genres:' in i.text:
            details[3] = str([r.text for r in i.find_all('span')][1:])
            pass
        if 'Theme:' in i.text or 'Themes:' in i.text:
            details[4] = str([r.text for r in i.find_all('span')][1:])
            pass
        if 'Demographic:' in i.text:
            details[5] = i.find('a').text
            pass
        if 'Popularity:' in i.text:
            details[6] = i.text.split('#')[1].strip()
            pass
    return details

In [5]:
%%time
details = []
for i in range(len(df)):
    res = requests.get(df.details[i])
    soup = BeautifulSoup(res.text, 'html.parser')
    details_page = soup.find('div', {'class': 'leftside'})
    details.append(get_details(details_page))
    if i % 500 == 0: print(f'{i+1}/{len(df)}')

1/50
CPU times: total: 10.8 s
Wall time: 1min 31s


In [6]:
df_details = pd.DataFrame(details, columns=['broadcast', 'studios', 'source','genres', 'themes', 'demographic', 'popularity'])
df_details.head()

,broadcast,studios,source,genres,themes,demographic,popularity
0,"['Sundays', 'at', '17:00', '(JST)']",Bones,Manga,"['Action', 'Adventure', 'Drama', 'Fantasy']",['Military'],Shounen,3
1,"['Saturdays', 'at', '00:00', '(JST)']",A-1 Pictures,Manga,"['Comedy', 'Romance']","['Psychological', 'School']",Seinen,242
2,"['Tuesdays', 'at', '00:00', '(JST)']",Pierrot,Manga,"['Action', 'Adventure', 'Fantasy']",NaN,Shounen,746
3,"['Wednesdays', 'at', '18:00', '(JST)']",Bandai Namco Pictures,Manga,"['Action', 'Comedy', 'Sci-Fi']","['Gag Humor', 'Historical', 'Parody', 'Samurai']",Shounen,337
4,"['Wednesdays', 'at', '02:05', '(JST)']",White Fox,Visual,"['Drama', 'Sci-Fi', 'Suspense']","['Psychological', 'Time Travel']",NaN,13


In [7]:
df_details.broadcast = df_details.broadcast.str.strip('[]').str.replace("'", '').str.replace(',', '')
df_details.genres = df_details.genres.str.strip('[]').str.replace("'", '')
df_details.themes = df_details.themes.str.strip('[]').str.replace("'", '')
df_details.head()

,broadcast,studios,source,genres,themes,demographic,popularity
0,Sundays at 17:00 (JST),Bones,Manga,"Action, Adventure, Drama, Fantasy",Military,Shounen,3
1,Saturdays at 00:00 (JST),A-1 Pictures,Manga,"Comedy, Romance","Psychological, School",Seinen,242
2,Tuesdays at 00:00 (JST),Pierrot,Manga,"Action, Adventure, Fantasy",NaN,Shounen,746
3,Wednesdays at 18:00 (JST),Bandai Namco Pictures,Manga,"Action, Comedy, Sci-Fi","Gag Humor, Historical, Parody, Samurai",Shounen,337
4,Wednesdays at 02:05 (JST),White Fox,Visual,"Drama, Sci-Fi, Suspense","Psychological, Time Travel",NaN,13


In [8]:
_df = df[:][:len(df_details)]
_df = pd.concat([_df, df_details], axis=1)
_df.sample()

,position,title,episodes,release_date,members,score,details,broadcast,studios,source,genres,themes,demographic,popularity
39,40,Mushishi Zoku Shou 2nd Season,TV (10 eps),Oct 2014 - Dec 2014,244829,8.74,https://myanimelist.net/anime/24701/Mushishi_Z...,Sundays at 00:30 (JST),Artland,Manga,"Adventure, Fantasy, Mystery, Slice of Life, Su...","Adult Cast, Historical, Iyashikei",Seinen,851


In [9]:
_df.to_csv('AllAnimesFull.csv', index=False)